In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from pyclustering.cluster.kmedoids import kmedoids
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from umap import UMAP
from sklearn.preprocessing import StandardScaler
import prince

import gower

%matplotlib inline
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

<IPython.core.display.Javascript object>

In [3]:
loans = pd.read_csv("data/sba_loans_cleaned3.csv")

<IPython.core.display.Javascript object>

In [4]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520954 entries, 0 to 520953
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   State               520954 non-null  object 
 1   BankState           520954 non-null  object 
 2   ApprovalFY          520954 non-null  int64  
 3   NoEmp               520954 non-null  int64  
 4   NewExist            520954 non-null  int64  
 5   RevLineCr           520421 non-null  float64
 6   LowDoc              516749 non-null  float64
 7   DisbursementGross   520954 non-null  float64
 8   MIS_Status          520954 non-null  int64  
 9   twoDigNAICS         520954 non-null  int64  
 10  is_franchise        520954 non-null  float64
 11  bank_out_of_state   520954 non-null  int64  
 12  Term_years          520954 non-null  float64
 13  job_category        520954 non-null  float64
 14  retained_category   520954 non-null  float64
 15  UrbanRural_cleaned  514461 non-nul

<IPython.core.display.Javascript object>

In [5]:
loans.head()
loans = loans.dropna()

<IPython.core.display.Javascript object>

In [6]:
drop_cols = [
    "ApprovalFY",
    "Disbr_Month_sin",
    "Disbr_Month_cos",
    "Appv_Month_sin",
    "Appv_Month_cos",
]
loans = loans.drop(columns=drop_cols)

<IPython.core.display.Javascript object>

In [7]:
loans["twoDigNAICS"].value_counts()

44    62620
72    58919
54    51748
81    49387
23    49170
62    35962
42    32022
56    27600
45    27319
33    24333
48    18080
32    12074
53    11326
71    10325
51     8879
52     8525
31     8260
61     5835
11     4151
49     1935
21     1058
22      409
55      132
92       80
0        11
Name: twoDigNAICS, dtype: int64

<IPython.core.display.Javascript object>

In [8]:
loans

,State,BankState,NoEmp,NewExist,RevLineCr,LowDoc,DisbursementGross,MIS_Status,twoDigNAICS,is_franchise,bank_out_of_state,Term_years,job_category,retained_category,UrbanRural_cleaned,Disbr_year,sba_pre_approv,percent_SBA,bank_size
0,NC,MN,2,1,0.0,0.0,253400.0,0,44,1.0,0,13.500000,0.0,0.0,0.0,2006,1.0,0.75,1.0
1,AR,MS,7,0,0.0,0.0,137300.0,0,72,0.0,0,10.500000,0.0,0.0,0.0,2006,1.0,0.85,0.0
2,CA,SD,18,1,1.0,0.0,438541.0,0,61,0.0,0,6.916667,1.0,2.0,0.0,2006,1.0,0.50,2.0
3,FL,FL,4,0,1.0,0.0,51440.0,0,23,0.0,1,7.000000,0.0,1.0,0.0,2006,1.0,0.50,2.0
4,LA,LA,3,0,0.0,0.0,50000.0,0,53,0.0,1,5.000000,0.0,0.0,0.0,2006,1.0,0.85,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520949,CO,IL,2,0,1.0,0.0,30000.0,0,61,0.0,0,7.000000,0.0,1.0,1.0,2006,1.0,0.50,2.0
520950,AZ,WI,2,1,0.0,0.0,170000.0,1,81,0.0,0,2.416667,1.0,1.0,0.0,2006,1.0,0.75,1.0
520951,NV,NV,20,0,0.0,0.0,509000.0,0,23,0.0,1,20.000000,2.0,0.0,0.0,2007,1.0,1.00,1.0
520952,AZ,AL,3,0,1.0,0.0,15635.0,0,51,0.0,0,7.000000,0.0,1.0,0.0,2006,1.0,0.50,2.0


<IPython.core.display.Javascript object>

In [9]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510160 entries, 0 to 520953
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   State               510160 non-null  object 
 1   BankState           510160 non-null  object 
 2   NoEmp               510160 non-null  int64  
 3   NewExist            510160 non-null  int64  
 4   RevLineCr           510160 non-null  float64
 5   LowDoc              510160 non-null  float64
 6   DisbursementGross   510160 non-null  float64
 7   MIS_Status          510160 non-null  int64  
 8   twoDigNAICS         510160 non-null  int64  
 9   is_franchise        510160 non-null  float64
 10  bank_out_of_state   510160 non-null  int64  
 11  Term_years          510160 non-null  float64
 12  job_category        510160 non-null  float64
 13  retained_category   510160 non-null  float64
 14  UrbanRural_cleaned  510160 non-null  float64
 15  Disbr_year          510160 non-nul

<IPython.core.display.Javascript object>

In [10]:
num_cols = ["NoEmp", "DisbursementGross", "Term_years", "Disbr_year", "percent_SBA"]
cat_cols = [
    "State",
    "BankState",
    # "MIS_Status",
    "twoDigNAICS",
    "NewExist",
    "RevLineCr",
    "LowDoc",
    "is_franchise",
    "bank_out_of_state",
    "UrbanRural_cleaned",
]

<IPython.core.display.Javascript object>

In [11]:
is_cat = loans.columns.isin(cat_cols)

<IPython.core.display.Javascript object>

In [12]:
nums = loans[num_cols]

<IPython.core.display.Javascript object>

In [13]:
sample = loans.sample(1000, random_state=42)

<IPython.core.display.Javascript object>

In [14]:
gower_dist = gower.gower_matrix(sample, cat_features=is_cat)

<IPython.core.display.Javascript object>

In [15]:
gower_df = pd.DataFrame(gower_dist, columns=sample.index, index=sample.index)

<IPython.core.display.Javascript object>

In [16]:
k = 5
nrows = gower_df.shape[0]
init_medoids = np.random.randint(0, nrows, k)
init_medoids

array([628, 262, 469, 360, 130])

<IPython.core.display.Javascript object>

In [17]:
kmed = kmedoids(
    gower_dist, initial_index_medoids=init_medoids, data_type="distance_matrix"
)

kmed.process()

<IPython.core.display.Javascript object>

In [ ]:
medoid_idxs = kmed.get_medoids()
medoid_idxs

labels = kmed.predict(gower_dist)

sample["label"] = labels

In [ ]:
sample.iloc[medoid_idxs, :]

In [ ]:
clst_avg = sample.groupby("label").mean().T
clst_avg.style.background_gradient(axis=1)

In [ ]:
sample.groupby("label")["NoEmp"].mean().plot.bar()

In [ ]:
loans["MIS_Status"].mean()

In [ ]:
sample.groupby("label")["MIS_Status"].mean()

In [ ]:
clst = DBSCAN(eps=0.2, min_samples=3, metric="precomputed")
clst.fit(gower_dist)

In [ ]:
# clst.labels_

In [ ]:
scaler = StandardScaler()
X_train_sub = sample[num_cols]


ns = [5, 10, 100]
for n in ns:
    umap = UMAP(n_neighbors=n, n_components=2)
    X_train_sub_umap = umap.fit_transform(X_train_sub)
    sns.scatterplot(
        X_train_sub_umap[:, 0],
        X_train_sub_umap[:, 1],
        hue=sample["MIS_Status"],
        alpha=0.25,
    )
    plt.title(f"UMAP, neighbors = {n}")
    plt.legend().remove()
    plt.show()

In [ ]:
X = loans.drop(columns=["MIS_Status"])
y = loans["MIS_Status"]

In [ ]:
X.isna().sum()
X["DisbursementGross"].describe()

In [ ]:
nums = X[num_cols]


In [ ]:
X.shape

In [ ]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X[num_cols])
X_std = pd.DataFrame(X_std, columns=num_cols, index=X[num_cols].index)
# pd.DataFrame(scaler.fit_transform(X[num_cols]), columns=num_cols)

scaled_X = pd.concat((X_std, X[cat_cols]), axis=1)
scaled_X

In [ ]:
X[cat_cols]

In [ ]:
famd = prince.FAMD(n_components=2)
famd.fit(scaled_X)

In [ ]:
famd.plot_row_coordinates(scaled_X, color_labels=X["LowDoc"], alpha=0.5)
plt.show()

# , color_labels=X["LowDoc"]

In [ ]:
famd.row_coordinates()

Re-engineered some of the data to have more continuous variables. 

In [ ]:
loans.head()

In [ ]:
loans2 = pd.read_csv("data/SBA_clnd.csv")
loans2.head()

In [ ]:
loans2 = loans2.drop(columns=drop_cols)
